<a href="https://colab.research.google.com/github/subhadarship/learning-to-unjumble/blob/master/mrpc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup working directory


In [0]:
import os

In [20]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
WORKING_DIR=os.path.join('/content', 'drive', 'My Drive', 'Colab Notebooks', 'NLU')
os.path.exists(WORKING_DIR)

True

In [22]:
cd $WORKING_DIR

/content/drive/My Drive/Colab Notebooks/NLU


# Clone transformers repo and checkout version 2.7.0

In [23]:
!rm -rf transformers
!git clone https://github.com/huggingface/transformers

Cloning into 'transformers'...
remote: Enumerating objects: 26386, done.
remote: Total 26386 (delta 0), reused 0 (delta 0), pack-reused 26386
Receiving objects: 100% (26386/26386), 15.87 MiB | 6.48 MiB/s, done.
Resolving deltas: 100% (18385/18385), done.
Checking out files: 100% (675/675), done.


In [24]:
cd transformers/

/content/drive/My Drive/Colab Notebooks/NLU/transformers


In [25]:
!git checkout -f v2.7.0

Checking out files: 100% (475/475), done.
Note: checking out 'v2.7.0'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 6f5a12a5 Release: v2.7.0


# Install packages required for running `run_glue.py`

In [26]:
!pip install -r ./examples/requirements.txt

In [27]:
!pip install boto3 filelock requests tqdm sentencepiece sacremoses tokenizers

# Download GLUE data

In [0]:
GLUE_DIR="/tmp/data"

In [29]:
!echo $GLUE_DIR

/tmp/data


In [30]:
!python ./utils/download_glue_data.py --help

usage: download_glue_data.py [-h] [--data_dir DATA_DIR] [--tasks TASKS]
                             [--path_to_mrpc PATH_TO_MRPC]

optional arguments:
  -h, --help            show this help message and exit
  --data_dir DATA_DIR   directory to save data to
  --tasks TASKS         tasks to download data for as a comma separated string
  --path_to_mrpc PATH_TO_MRPC
                        path to directory containing extracted MRPC data,
                        msr_paraphrase_train.txt and msr_paraphrase_text.txt


In [31]:
!python ./utils/download_glue_data.py --data_dir $GLUE_DIR --tasks MRPC

Processing MRPC...
Local MRPC data not specified, downloading data from https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_train.txt
	Completed!


# Compute RoBERTa score on MRPC task

In [0]:
TASK_NAME="MRPC"

In [33]:
!echo $TASK_NAME

MRPC


In [34]:
!echo $PYTHONPATH

/content/drive/My Drive/Colab Notebooks/NLU/transformers/src:/env/python


In [35]:
%set_env PYTHONPATH=$WORKING_DIR/transformers/src:/env/python

env: PYTHONPATH=/content/drive/My Drive/Colab Notebooks/NLU/transformers/src:/env/python


In [36]:
!echo $PYTHONPATH

/content/drive/My Drive/Colab Notebooks/NLU/transformers/src:/env/python


In [47]:
MODEL_DIR = os.path.join('/content', 'drive', 'My Drive', 'Colab Notebooks', 'NLU', 'models', 'random_0.15')
os.path.exists(MODEL_DIR)

True

In [52]:
!python ./examples/run_glue.py \
    --model_type roberta \
    --model_name_or_path /content/drive/My\ Drive/Colab\ Notebooks/NLU/models/random_0.15 \
    --task_name $TASK_NAME \
    --do_train \
    --do_eval \
    --data_dir $GLUE_DIR/$TASK_NAME \
    --max_seq_length 128 \
    --per_gpu_eval_batch_size=64   \
    --per_gpu_train_batch_size=64   \
    --learning_rate 1e-4 \
    --num_train_epochs 3 \
    --output_dir glue_model \
    --overwrite_output_dir

2020-05-15 02:39:31.472607: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
05/15/2020 02:39:33 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
05/15/2020 02:39:34 - INFO - transformers.configuration_utils -   loading configuration file /content/drive/My Drive/Colab Notebooks/NLU/models/random_0.15/config.json
05/15/2020 02:39:34 - INFO - transformers.configuration_utils -   Model config RobertaConfig {
  "_num_labels": 2,
  "architectures": [
    "RobertaForTokenDiscrimination"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": 2,
  "finetuning_task": "mrpc",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermed